In [1]:
from threading import Thread
from queue import Queue, Empty
from subprocess import Popen, PIPE
from time import sleep

In [2]:
class NonBlockingStreamReader:

    def __init__(self, stream):
        '''
        stream: the stream to read from.
                Usually a process' stdout or stderr.
        '''

        self._s = stream
        self._q = Queue()

        def _populateQueue(stream, queue):
            '''
            Collect lines from 'stream' and put them in 'quque'.
            '''

            while True:
                line = stream.readline()
                if line:
                    queue.put(line)
                else:
                    raise UnexpectedEndOfStream

        self._t = Thread(target = _populateQueue,
                args = (self._s, self._q))
        self._t.daemon = True
        self._t.start() #start collecting lines from the stream

    def readline(self, timeout = None):
        try:
            return self._q.get(block = timeout is not None,
                    timeout = timeout)
        except Empty:
            return None

class UnexpectedEndOfStream(Exception): pass

def output_from_command(command=None):
    if command:
        p.stdin.write(command)
        p.stdin.flush()
    outputList = []
    while True:
        output = nbsr.readline(0.1) # 0.1 secs to let the shell output the result
        if not output:
            outputList.append('No more data\n\n')
            return outputList
        else:
            outputList.append(output)

In [3]:
#p = Popen(["/home/ubuntu/.opam/4.06.1/bin/sertop","--printer=human"], stdin=PIPE, stdout=PIPE, shell=True)
p = Popen(["/home/ubuntu/.opam/4.06.1/bin/sertop"], stdin=PIPE, stdout=PIPE, shell=True)
nbsr = NonBlockingStreamReader(p.stdout)
for i in output_from_command(None):
    print(i)

b'(Feedback((doc_id 0)(span_id 0)(route 0)(contents(FileLoaded Coq.Init.Prelude /home/ubuntu/.opam/4.06.1/lib/coq/theories/Init/Prelude.vo))))\n'
b'(Feedback((doc_id 0)(span_id 0)(route 0)(contents(FileLoaded Coq.Init.Notations /home/ubuntu/.opam/4.06.1/lib/coq/theories/Init/Notations.vo))))\n'
b'(Feedback((doc_id 0)(span_id 0)(route 0)(contents(FileLoaded Coq.Init.Logic /home/ubuntu/.opam/4.06.1/lib/coq/theories/Init/Logic.vo))))\n'
b'(Feedback((doc_id 0)(span_id 0)(route 0)(contents(FileLoaded Coq.Init.Datatypes /home/ubuntu/.opam/4.06.1/lib/coq/theories/Init/Datatypes.vo))))\n'
b'(Feedback((doc_id 0)(span_id 0)(route 0)(contents(FileLoaded Coq.Init.Logic_Type /home/ubuntu/.opam/4.06.1/lib/coq/theories/Init/Logic_Type.vo))))\n'
b'(Feedback((doc_id 0)(span_id 0)(route 0)(contents(FileLoaded Coq.Init.Specif /home/ubuntu/.opam/4.06.1/lib/coq/theories/Init/Specif.vo))))\n'
b'(Feedback((doc_id 0)(span_id 0)(route 0)(contents(FileLoaded Coq.Init.Decimal /home/ubuntu/.opam/4.06.1/lib/coq/th

In [4]:
commandList = [b'(Add () "Lemma addn0 n : n + 0 = n. Proof. now induction n. Qed.")\n',
               b'(Exec 5)',
               b'(Query ((sid 3)) Goals)',
               b'(Query ((sid 3) (pp ((pp_format PpStr)))) Goals)',
               b'(Query ((sid 4)) Ast)',
               b'(pp_ex (Print () (CoqConstr (App (Rel 0) ((Rel 0))))))',
               b'(Query () (Vernac "Print nat. "))',
               b'(Query () (Definition nat))']

In [5]:
command = commandList[7]

print(command)
print()
for i in output_from_command(command=command):
    print(i)

b'(Query () (Definition nat))'

b'(Feedback((doc_id 0)(span_id 1)(route 0)(contents(ProcessingIn master))))\n'
b'(Feedback((doc_id 0)(span_id 1)(route 0)(contents Processed)))\n'
b'(Answer 0 Ack)\n'
No more data




Exception in thread Thread-4:
Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/ubuntu/anaconda3/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-2-ba18f13ddfe7>", line 22, in _populateQueue
    raise UnexpectedEndOfStream
UnexpectedEndOfStream

